# Pandas - a quick introduction

In this tutorial, we'll explore pandas, the Python data analysis toolkit. Our emphasis will be on working with dataframes, the primary pandas object. Although we don't have time to study all the features of dataframes, extensive documentation can be found here https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html

Let's dive into an Olympic Medal dataset available from Wikipedia https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table. I've already done a little bit of cleanup so that we can quickly get to the important features.

In [ ]:
import pandas as pd

Our file is in csv format, so we'll use the read_csv method. We know that the first two rows contain comments and other data that we won't want. We can use the skiprows argument to skip over these rows. We'll set index_col to zero so that the first column serves and the index.

In [ ]:
df = pd.read_csv('olympics.csv', index_col=0, skiprows=2)

In [ ]:
df.head()

In [ ]:
df.tail()

Note that the last row of our dataframe contains totals for the number of games and medals won. Let's get rid of that using the drop method.

In [ ]:
df = df.drop('Totals')
df.tail()

We can manipulate the data in our data frame. For example, let's cleanup the country names to get rid of everything from the country abbreviation to the end of the string. For example "Australia (AUS) [AUS] [Z]" becomes simply "Australia".

We'll start by calling the str.split method to split the country names on the opening parenthesis "(" and then reassign the first element of the resulting list to the index. We then display the head of the dataframe to confirm that the renaming of the countries worked as expected.

In [ ]:
names_ids = df.index.str.split('\s\(')
df.index = names_ids.str[0]
df.head()

The columns attribute returns the column names. Note that we don't return the first column, which was already assigned as the index.

In [ ]:
df.columns

Rows are accessed by row number using the iloc. In the example below, we pull out the data for Argentina

In [ ]:
df.iloc[2]

We can also access a row by the label or value of the index. For example, if we wanted to get the data for France, it's more convenient to use the label than figure out the row number.

In [ ]:
df.loc['France']

Columns are accessed using the column names. In the example below, we return a series containing the number of summer gold medals by country.

In [ ]:
df['Summer silver'].head()

We can select multiple columns from a dataframe by passing a list of column names rather than a single name

In [ ]:
df[['Summer gold', 'Summer silver', 'Summer bronze']].head()

If we just want the index, this is accessed using the index attribute

In [ ]:
df.index

We can add columns to our dataframe. For example, we'll calculate a weighted total for combined medals where gold=3, silver=2 and bronze=1.

In [ ]:
df['Combined weighted'] = df['Combined gold']*3 + df['Combined silver']*2 + df['Combined bronze']

In [ ]:
df['Combined weighted'].head()

Pandas provides methods for finding min and max values and the corresponding index. Below, we find the country that won the most winter gold medals and the number that they won.

In [ ]:
print(df['Winter gold'].max())
print(df['Winter gold'].idxmax())

We can select rows from the dataframe based on the values in a column. In the example below, we filter on countries that have won at least 50 gold medals in the winter Olympics.

In [ ]:
df.loc[ df['Winter gold']>50 ]

We can filter on multiple columns. In the example below, we limit the output to countries that also won more than 50 summer gold medals and assign the results to a new dataframe

In [ ]:
df2 = df.loc[ (df['Winter gold']>50) & (df['Summer gold']>50) ]
df2

Let's create a simpler dataframe that is limited to the gold medal results and then use the sum method to sum the values in a column.

In [ ]:
df2 = df2[['Summer gold', 'Winter gold']]
df2

In [ ]:
df2['Winter gold'].sum()

## Missing values

Observation data often has missing values, which can lead to problems with their analysis. Fortunately, pandas provides fillna and interpolate methods for filling in these missing values (data imputation)

Let's explore this using a small data set containing temperatures in US cities over the course of a week.

In [ ]:
# Missing values are dislayed as NaNs (Not a Number)
df3 = pd.read_csv('city temps.csv', index_col=0, skiprows=0)
df3

In [ ]:
# The simplest method is to replace missing values with a fixed value
df3.fillna(value=70)

In [ ]:
# Forward filling using the last valid value to fill missing value
# Note that in this case we used axis=1 so that we propogate across
# the rows rather than the values

df3.fillna(method="ffill", axis=1)

In [ ]:
# Back filling using the NEXT valid value to fill missing value
# Note that in this case we used axis=1 so that we propogate across
# the rows rather than the values

df3.fillna(method="bfill", axis=1)

In [ ]:
# As of version 0.17.0, pandas provides an interpolate function that fills missing values
# By default, we get a linear interpolation, but a number of other options are available
# (quadratic, cubic, polynomial, etc.)

df3.interpolate(method="linear", axis=1)

## Data formats

Until now, we've been working with csv files, but pandas can handle many other formats including json, html, excel and HDF. This is extremely useful since we don't need to create csv files from richer data formats. We show an example below where we read two sheets from an Excel file

In [ ]:
df4 = pd.read_excel('city temps spreadsheet.xlsx', index_col=0, skiprows=0, sheetname='set1')
df4

In [ ]:
df5 = pd.read_excel('city temps spreadsheet.xlsx', index_col=0, skiprows=0, sheetname='set2')
df5